Обнаружение и ликвидация дубликатов

In [5]:
import pandas as pd
import numpy as np

sber_data=pd.read_csv('data/sber_data.csv', sep=',')
sber_data['id'].nunique() == sber_data.shape[0]

# метод dublicated(). параметры : SUBSET - список признаков, по которым производится поиск дубликатов
# Найдём число полных дубликатов таблице sber_data. Предварительно создадим список столбцов dupl_columns, 
# по которым будем искать совпадения (все столбцы, не включая id). Создадим маску дубликатов с помощью 
# метода duplicated() и произведём фильтрацию. Результат заносим в переменную sber_duplicates. 
# Выведем число строк в результирующем DataFrame:
dupl_columns = list(sber_data.columns)
dupl_columns.remove('id')

mask = sber_data.duplicated(subset=dupl_columns)
sber_duplicates = sber_data[mask]
print(f'Число найденных дубликатов: {sber_duplicates.shape[0]}')
# 562 дубликата

# Теперь нам необходимо от них избавиться. Для этого легче всего воспользоваться методом drop_duplicates(), 
# который удаляет повторяющиеся записи из таблицы. Создадим новую таблицу sber_dedupped, которая будет 
# версией исходной таблицы, очищенной от полных дубликатов.

sber_dedupped = sber_data.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {sber_dedupped.shape[0]}')

Число найденных дубликатов: 562
Результирующее число записей: 29909


НЕИНФОРМАТИВНЫЕ ПРИЗНАКИ

In [7]:
# Существует правило 95 или 99. Если признак содержит 95 или 99 уникальных элементов или же одинаковых - этот признак 
# неинформативан. Лучше на основе его не строить модель

# Создаём пустой список low_information_cols, куда будем добавлять названия признаков, которые мы посчитаем неинформативными.
# В цикле пройдёмся по всем именам столбцов в таблице и для каждого будем совершать следующие действия: рассчитаем 
# top_freq — наибольшую относительную частоту с помощью метода value_counts() с параметром normalize=True. 
# Метод вернёт долю от общих данных, которую занимает каждое уникальное значение в признаке
# рассчитаем nunique_ratio — отношение числа уникальных значений в столбце к размеру всего столбца. 
# Число уникальных значений в столбце получим с помощью метода nunique(), а размер признака — с помощью метода count(). 
# Например, для столбца id число уникальных значений — 30471; оно же равно размеру таблицы. 
# Поэтому результат отношения будет 1. Сравним каждое из полученных чисел с пороговым значением (у нас это 0.95) 
# и добавим в список неинформативных признаков, если условие истинно.
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in sber_data.columns:
    #наибольшая относительная частота в признаке
    top_freq = sber_data[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = sber_data[col].nunique() / sber_data[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

# дропнем весь этот мусор через drop
information_sber_data = sber_data.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_sber_data.shape[1]}')

id: 100.0% уникальных значений
oil_chemistry_raion: 99.03% одинаковых значений
railroad_terminal_raion: 96.27% одинаковых значений
nuclear_reactor_raion: 97.17% одинаковых значений
big_road1_1line: 97.44% одинаковых значений
mosque_count_1000: 98.08% одинаковых значений
Результирующее число признаков: 55


ВАЖНОСТЬ ПРИЗНАКОВ

In [ ]:
# На самом деле информативность признаков определяется не только числом уникальных значений, но и их влиянием на 
# целевой признак (тот, который мы хотим предсказать). Это называется важностью признака. 

# Признаки, которые обладают низкой важностью, называют нерелевантными признаками. Например, если бы в наших данных 
# о квартирах был признак, содержащий информацию о температуре воздуха за окном, он был бы нерелевантным. 
# Нерелевантные признаки могут быть не такими очевидными. В таких случаях применяются более сложные способы их поиска 
# с использованием статистики и моделей машинного обучения. Мы поговорим об этом позже, когда вплотную 
# подойдём к Machine Learning.